# Your First AI application


Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 


In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) from Oxford of 102 flower categories, you can see a few examples below. 


<img src='assets/Flowers.png' width=500px>


The project is broken down into multiple steps:


* Load the image dataset and create a pipeline.

* Build and Train an image classifier on this dataset.

* Use your trained model to perform inference on flower images.


We'll lead you through each part which you'll implement in Python.


When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

## Import Resources

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image

ModuleNotFoundError: No module named 'tensorflow'

## Load the Dataset


Here you'll use `tensorflow_datasets` to load the [Oxford Flowers 102 dataset](https://www.tensorflow.org/datasets/catalog/oxford_flowers102). This dataset has 3 splits: `'train'`, `'test'`, and `'validation'`.  You'll also need to make sure the training data is normalized and resized to 224x224 pixels as required by the pre-trained networks.


The validation and testing sets are used to measure the model's performance on data it hasn't seen yet, but you'll still need to normalize and resize the images to the appropriate size.

In [2]:
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

train_ds = dataset['train']
val_ds = dataset['validation']
test_ds = dataset['test']

NameError: name 'tfds' is not defined

## Explore the Dataset

In [3]:
print(f"Number of training examples: {info.splits['train'].num_examples}")
print(f"Number of validation examples: {info.splits['validation'].num_examples}")
print(f"Number of test examples: {info.splits['test'].num_examples}")

NameError: name 'info' is not defined

In [4]:
num_classes = info.features['label'].num_classes
print(f"Number of classes: {num_classes}")

NameError: name 'info' is not defined

In [5]:
for image, label in train_ds.take(3):
    print(f"Image shape: {image.shape}, Label: {label}")

NameError: name 'train_ds' is not defined

In [6]:
image, label = next(iter(train_ds))
plt.figure()
plt.imshow(image)
plt.title(label)
plt.axis('off')
plt.show()

NameError: name 'train_ds' is not defined

### Label Mapping


You'll also need to load in a mapping from label to category name. You can find this in the file `label_map.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/3.7/library/json.html). This will give you a dictionary mapping the integer coded labels to the actual names of the flowers.

In [7]:
with open('label_map.json', 'r') as f:
    class_names = json.load(f)

image, label = next(iter(train_ds))
plt.figure()
plt.imshow(image)
plt.title(class_names[str(label.numpy())])
plt.axis('off')
plt.show()

NameError: name 'json' is not defined

## Create Pipeline

In [8]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (224, 224))
    return image, label

batch_size = 32
train_ds = train_ds.map(preprocess).cache().shuffle(1000).batch(batch_size)
val_ds = val_ds.map(preprocess).cache().batch(batch_size)
test_ds = test_ds.map(preprocess).cache().batch(batch_size)

NameError: name 'train_ds' is not defined

# Build and Train the Classifier


Now that the data is ready, it's time to build and train the classifier. You should use the MobileNet pre-trained model from TensorFlow Hub to get the image features. Build and train a new feed-forward classifier using those features.


We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! 


Refer to the rubric for guidance on successfully completing this section. Things you'll need to do:


* Load the MobileNet pre-trained network from TensorFlow Hub.

* Define a new, untrained feed-forward network as a classifier.

* Train the classifier.

* Plot the loss and accuracy values achieved during training for the training and validation set.

* Save your trained model as a Keras model. 


We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!


When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right.


**Note for Workspace users:** One important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module. Also, If your model is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [9]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224, 3))
feature_extractor.trainable = False

NameError: name 'hub' is not defined

In [10]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes)
])

NameError: name 'tf' is not defined

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)

NameError: name 'model' is not defined

In [12]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

NameError: name 'model' is not defined

In [13]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined

## Testing your Network


It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [14]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

NameError: name 'model' is not defined

## Save the Model


Now that your network is trained, save the model so you can load it later for making inference. In the cell below save your model as a Keras model (*i.e.* save it as an HDF5 file).

In [15]:
model.save('my_model.h5')

NameError: name 'model' is not defined

## Load the Keras Model


Load the Keras model you saved above.

In [16]:
loaded_model = tf.keras.models.load_model('my_model.h5')

NameError: name 'tf' is not defined

# Inference for Classification


Now you'll write a function that uses your trained network for inference. Write a function called `predict` that takes an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 


```python

probs, classes = predict(image_path, model, top_k)

```


If `top_k=5` the output of the `predict` function should be something like this:


```python

probs, classes = predict(image_path, model, 5)

print(probs)

print(classes)

> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]

> ['70', '3', '45', '62', '55']

```


Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.


The `predict` function will also need to handle pre-processing the input image such that it can be used by your model. We recommend you write a separate function called `process_image` that performs the pre-processing. You can then call the `process_image` function from the `predict` function. 

### Image Pre-processing


The `process_image` function should take in an image (in the form of a NumPy array) and return an image in the form of a NumPy array with shape `(224, 224, 3)`.


First, you should convert your image into a TensorFlow Tensor and then resize it to the appropriate size using `tf.image.resize`.


Second, the pixel values of the input images are typically encoded as integers in the range 0-255, but the model expects the pixel values to be floats in the range 0-1. Therefore, you'll also need to normalize the pixel values. 


Finally, convert your image back to a NumPy array using the `.numpy()` method.

In [17]:
def process_image(image):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, [224, 224])
    return image.numpy()

In [18]:
image_path = './test_images/hard-leaved_pocket_orchid.jpg'
im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)

fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(processed_test_image)
ax2.set_title('Processed Image')
plt.tight_layout()
plt.show()

NameError: name 'Image' is not defined

Once you can get images in the correct format, it's time to write the `predict` function for making inference with your model.


### Inference


Remember, the `predict` function should take an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 


```python

probs, classes = predict(image_path, model, top_k)

```


If `top_k=5` the output of the `predict` function should be something like this:


```python

probs, classes = predict(image_path, model, 5)

print(probs)

print(classes)

> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]

> ['70', '3', '45', '62', '55']

```


Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.


**Note:** The image returned by the `process_image` function is a NumPy array with shape `(224, 224, 3)` but the model expects the input images to be of shape `(1, 224, 224, 3)`. This extra dimension represents the batch size. We suggest you use the `np.expand_dims()` function to add the extra dimension. 

In [19]:
def predict(image_path, model, top_k=5):
    image = Image.open(image_path)
    image = np.asarray(image)
    processed = process_image(image)
    processed = np.expand_dims(processed, 0)
    predictions = model.predict(processed)
    top_indices = np.argsort(predictions[0])[-top_k:][::-1]
    top_probs = tf.nn.softmax(predictions[0])[0][top_indices]
    classes = [str(i) for i in top_indices]
    return top_probs, classes

# Sanity Check


It's always good to check the predictions made by your model to make sure they are correct. To check your predictions we have provided 4 images in the `./test_images/` folder:


* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg


In the cell below use `matplotlib` to plot the input image alongside the probabilities for the top 5 classes predicted by your model. Plot the probabilities as a bar graph. The plot should look like this:


<img src='assets/inference_example.png' width=600px>


You can convert from the class integer labels to actual flower names using `class_names`. 

In [20]:
image_path = './test_images/hard-leaved_pocket_orchid.jpg'
probs, classes = predict(image_path, loaded_model, 5)

with open('label_map.json', 'r') as f:
    class_names = json.load(f)

fig, (ax1, ax2) = plt.subplots(figsize=(10, 5), ncols=2)
image = Image.open(image_path)
ax1.imshow(image)
ax1.axis('off')
ax1.set_title('Input Image')

y_pos = np.arange(len(classes))
ax2.barh(y_pos, probs)
ax2.set_yticks(y_pos)
ax2.set_yticklabels([class_names[cls] for cls in classes])
ax2.set_xlabel('Probability')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

NameError: name 'loaded_model' is not defined